In [1]:
#BostonHouseValues-tf.estimator
#we'll write an input function for preprocessing a subset of Boston housing data pulled from the UCI Housing Data Set and use it to feed data to a neural network regressor for predicting median house values.


In [3]:
import itertools

import pandas as pd
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
#Define the column names for the data set in COLUMNS. To distinguish features from the label, also define FEATURES and LABEL. Then read the three CSVs (tf.train, tf.test, and predict) into pandas DataFrames:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age","dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm","age", "dis", "tax", "ptratio"]
LABEL = "medv"

In [11]:
training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)
#training_set.head(10)
#test_set.head(10)
prediction_set.head(10)


,crim,zn,indus,nox,rm,age,dis,tax,ptratio,medv
0,0.03359,75.0,2.95,0.428,7.024,15.8,5.4011,252,18.3,NaN
1,5.09017,0.0,18.10,0.713,6.297,91.8,2.3682,666,20.2,NaN
2,0.12650,25.0,5.13,0.453,6.762,43.4,7.9809,284,19.7,NaN
3,0.05515,33.0,2.18,0.472,7.236,41.1,4.0220,222,18.4,NaN
4,8.15174,0.0,18.10,0.700,5.390,98.9,1.7281,666,20.2,NaN
5,0.24522,0.0,9.90,0.544,5.782,71.7,4.0317,304,18.4,NaN


In [14]:
#Defining FeatureColumns and Creating the Regressor
#create a list of FeatureColumns for the input data, which formally specify the set of features to use for training.
#all features in the housing data set contain continuous values, you can create their FeatureColumns using the tf.contrib.layers.real_valued_column() function:

feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]
#instantiate a DNNRegressor for the neural network regression model. 
#You'll need to provide two arguments here: hidden_units, a hyperparameter specifying the number of nodes in each hidden layer 
#(here, two hidden layers with 10 nodes each), and feature_columns, 
#containing the list of FeatureColumns you just defined
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,hidden_units=[10, 10],model_dir="/tmp/boston_model")
#Building the input_fn
#To pass input data into the regressor, write a factory method that accepts a pandas Dataframe and returns an input_fn:
def get_input_fn(data_set, num_epochs=None, shuffle=True):
    
    return tf.estimator.inputs.pandas_input_fn(x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
                                             y = pd.Series(data_set[LABEL].values),
                                             num_epochs=num_epochs,
                                             shuffle=shuffle)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/boston_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [15]:
#Training the Regressor
regressor.train(input_fn=get_input_fn(training_set), steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/boston_model\model.ckpt.
INFO:tensorflow:loss = 65181.9, step = 1
INFO:tensorflow:global_step/sec: 213.754
INFO:tensorflow:loss = 12071.7, step = 101 (0.469 sec)
INFO:tensorflow:global_step/sec: 228.051
INFO:tensorflow:loss = 9339.46, step = 201 (0.378 sec)
INFO:tensorflow:global_step/sec: 282.903
INFO:tensorflow:loss = 9458.73, step = 301 (0.374 sec)
INFO:tensorflow:global_step/sec: 286.433
INFO:tensorflow:loss = 6145.79, step = 401 (0.329 sec)
INFO:tensorflow:global_step/sec: 289.915
INFO:tensorflow:loss = 5803.35, step = 501 (0.361 sec)
INFO:tensorflow:global_step/sec: 288.331
INFO:tensorflow:loss = 9085.06, step = 601 (0.337 sec)
INFO:tensorflow:global_step/sec: 258.305
INFO:tensorflow:loss = 6616.0, step = 701 (0.390 sec)
INFO:tensorflow:global_step/sec: 278.28
INFO:tensorflow:loss = 6990.19, step = 801 (0.371 sec)
INFO:tensorflow:global_step/sec: 292.195
INFO:tensorflow:loss = 7876.19, 

In [16]:
#Evaluating the Model
ev = regressor.evaluate(input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))
#retrive loss
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

INFO:tensorflow:Starting evaluation at 2017-09-30-05:08:05
INFO:tensorflow:Restoring parameters from /tmp/boston_model\model.ckpt-5000
INFO:tensorflow:Finished evaluation at 2017-09-30-05:08:05
INFO:tensorflow:Saving dict for global step 5000: average_loss = 12.2721, global_step = 5000, loss = 1227.21
Loss: 1227.209839


In [17]:
#Making Predictions

y = regressor.predict(input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False))
# .predict() returns an iterator of dicts; convert to a list and print
# predictions
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Restoring parameters from /tmp/boston_model\model.ckpt-5000
Predictions: [array([ 34.77877045], dtype=float32), array([ 19.55076027], dtype=float32), array([ 23.80444717], dtype=float32), array([ 36.26771927], dtype=float32), array([ 15.91741848], dtype=float32), array([ 19.43694878], dtype=float32)]
